In [1]:
import os
import pandas as pd
import pymongo


import requests
from requests_oauthlib import OAuth1
import json

from __future__ import print_function

/Users/evgenyp/Projects/AmzJet-ZT/amzjet-backend/amz-deploy/dev/.venv-dev/lib/python2.7/site-packages/OpenSSL/crypto.py:14: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import utils, x509


In [2]:
DB_NAME = 'amz_local_search'
MONGO_URI = os.environ.get('AMZ_MONGO_URI_TEMPLATE') % (os.environ.get('AMZ_MONGO_PASSWORD'), DB_NAME)

client = pymongo.MongoClient(MONGO_URI)
db = client.get_database()

In [3]:
OUT_DIR = '.activities/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

# Load WooCommerce customers

In [14]:
WC_DAGA_DIR = '.woo_data/'

with open(os.path.join(WC_DAGA_DIR, 'wc_customers.json'), 'r') as f:
    raw_wc_customers = json.load(f)
len(raw_wc_customers)

1559

In [15]:
def enum_wc_cusotmers():
    for e in raw_wc_customers:
        yield {
            'wc_user_id': e.get('id'),
            'wc_email': e.get('email'),
            'wc_user_created': e.get('date_created'),
            'wc_is_paying': e.get('is_paying_customer'),
            'wc_role': e.get('role')            
        }
wc_customers_df = pd.DataFrame(enum_wc_cusotmers())
wc_customers_df.head(1)

,wc_email,wc_is_paying,wc_role,wc_user_created,wc_user_id
0,138213802@qq.com,False,customer,2020-04-13T20:34:01,17874


In [16]:
print ('wc_customers_df.is_paying_customer=True: ', len(wc_customers_df[wc_customers_df['wc_is_paying']]))

wc_customers_df.is_paying_customer=True:  264


In [17]:
wc_customers_df[wc_customers_df['wc_user_id'] == 17530]

,wc_email,wc_is_paying,wc_role,wc_user_created,wc_user_id


# Load WooCommerce orders

In [18]:
with open(os.path.join(WC_DAGA_DIR, 'wc_orders.json'), 'r') as f:
    raw_wc_orders = json.load(f)
len(raw_wc_orders)

1718

In [19]:
def enum_wc_orders():
    for order in raw_wc_orders:
        for line_item in order.get('line_items', []):
            yield {
                'wc_user_id': order.get('customer_id'),
                'wc_ip_address': order.get('customer_ip_address'),
                'wc_order_total': float(order.get('total')),
                'wc_item_total': float(line_item.get('total')),
                'wc_product': line_item.get('name')
            }
wc_order_items_df = pd.DataFrame(enum_wc_orders())
wc_order_items_df.head(2)

,wc_ip_address,wc_item_total,wc_order_total,wc_product,wc_user_id
0,142.167.30.70,15.0,15.0,Subscription - Starter,18062
1,50.48.1.241,0.0,0.0,Subscription - Free,18009


In [20]:
wc_orders_df = wc_order_items_df.groupby('wc_user_id').agg({
        'wc_ip_address': { 'wc_ip_address': 'last' },
        'wc_product': { 'wc_products': set }, 
        'wc_item_total': { 'wc_item_total': 'sum', 'wc_payments': list }
    })

wc_orders_df.columns = wc_orders_df.columns.droplevel(0)
wc_orders_df['wc_products'] = wc_orders_df['wc_products'].map(', '.join)
wc_orders_df['wc_payments'] = wc_orders_df['wc_payments'].map(str)
wc_orders_df.sort_values(['wc_item_total'], ascending=[False], inplace=True)
wc_orders_df.head(10)

/Users/evgenyp/Projects/AmzJet-ZT/amzjet-backend/amz-deploy/dev/.venv-dev/lib/python2.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,wc_products,wc_item_total,wc_payments,wc_ip_address
wc_user_id,,,,
17436,Audit - 1-3M,3000.00,[3000.0],107.77.231.4
17413,"Feature: Volatility, variability, Subscription...",2093.00,"[200.0, 49.0, 200.0, 49.0, 200.0, 49.0, 200.0,...",136.49.90.82
17827,"Feature: ASINs included in keyword - 10 ASINs,...",1825.00,"[858.0, 858.0, 10.0, 99.0]",12.186.143.218
17500,"Subscription - Basic, Feature: ASINs included ...",1154.50,"[21.25, 8.5, 21.25, 8.5, 21.25, 8.5, 21.25, 8....",74.88.44.149
0,"Feature: Rank decimal places (new), Feature: A...",1153.75,"[15.0, 15.0, 15.0, 0.5, 0.25, 15.0, 15.0, 15.0...",73.109.231.65
17849,"Subscription - Premium, Feature: Rank decimal ...",1079.00,"[100.0, 0.0, 100.0, 0.0, 100.0, 0.0, 100.0, 0....",172.250.11.175
17650,"Feature: Rank decimal places (new), Subscripti...",957.00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",115.189.88.197
18211,Subscription (Annual) - Premium,858.00,[858.0],174.195.207.30
17493,"Subscription - Basic, Feature: ASINs included ...",805.00,"[25.0, 10.0, 25.0, 10.0, 25.0, 10.0, 25.0, 10....",76.27.122.206


# Fetch data from Actions collection into a DataFrame

In [23]:
raw_actions_df = pd.DataFrame(db.actions.find({}))
raw_actions_df.head(1)

,_id,action_sig,args,occurred_at,stat,state,tracking,updated_at,user
0,1-S32M4kGRT6ps7uUJzKygVg,amz_local_search.search,"{u'query': u'coloring book', u'use_dummy_searc...",2019-10-09 04:09:44.605,NaN,{u'done': False},"{u'ip': u'10.32.0.1', u'ua': u'Mozilla/5.0 (Ma...",NaT,{u'id': 1}


In [25]:
actions_df = raw_actions_df[['action_sig', 'occurred_at', 'updated_at']].copy(deep=False)
actions_df['action_id'] = raw_actions_df['_id']
actions_df['user_id'] = raw_actions_df['user'].map(lambda u: u.get('id'))
actions_df['succeeded'] = raw_actions_df['state'].map(lambda s: s.get('success'))
actions_df['done'] = raw_actions_df['state'].map(lambda s: s.get('done'))
actions_df['arg_query'] = raw_actions_df['args'].map(lambda a: a.get('query'))
actions_df['arg_asin'] = raw_actions_df['args'].map(lambda a: a.get('asin'))
actions_df['error_type'] = raw_actions_df['state'].map(lambda a: a.get('error', {}).get('type_id'))
actions_df['error_msg'] = raw_actions_df['state'].map(lambda a: a.get('error', {}).get('msg'))

actions_df['occurred_at_day'] = actions_df['occurred_at'].dt.to_period('D')
actions_df['occurred_at_week'] = actions_df['occurred_at'].dt.to_period('W')
actions_df['occurred_at_month'] = actions_df['occurred_at'].dt.to_period('M')

actions_df.head(5)

,action_sig,occurred_at,updated_at,action_id,user_id,succeeded,done,arg_query,arg_asin,error_type,error_msg,occurred_at_day,occurred_at_week,occurred_at_month
0,amz_local_search.search,2019-10-09 04:09:44.605,NaT,1-S32M4kGRT6ps7uUJzKygVg,1,None,False,coloring book,None,None,None,2019-10-09,2019-10-07/2019-10-13,2019-10
1,amz_local_search.search,2019-10-09 04:21:46.943,2019-10-09 04:22:44.631,1-h63xBWR6iTuArrR8QPUtAa,1,True,True,coloring book,None,None,None,2019-10-09,2019-10-07/2019-10-13,2019-10
2,amz_local_search.search,2019-10-09 04:23:43.985,2019-10-09 04:24:33.907,1-EzTJ8DAoYaY4UcUQDUFgH8,1,True,True,carabiner clip,None,None,None,2019-10-09,2019-10-07/2019-10-13,2019-10
3,amz_local_search.search,2019-10-09 04:25:51.459,2019-10-09 04:26:28.723,1-6ZA5XxhiNK85SZv2JSUUiG,1,True,True,bath toy organizer,None,None,None,2019-10-09,2019-10-07/2019-10-13,2019-10
4,amz_local_search.search,2019-10-09 05:28:57.315,2019-10-09 05:29:57.280,2-BxRGqusHNrSPogSRncqo8B,2,True,True,sonic electric toothbrush,None,None,None,2019-10-09,2019-10-07/2019-10-13,2019-10


# Join actions with other metadata

In [26]:
full_actions_df = actions_df.merge(wc_customers_df, how='left', left_on='user_id', right_on='wc_user_id')
full_actions_df = full_actions_df.merge(wc_orders_df, how='left', left_on='user_id', right_on='wc_user_id')
full_actions_df.head(1)

,action_sig,occurred_at,updated_at,action_id,user_id,succeeded,done,arg_query,arg_asin,error_type,...,occurred_at_month,wc_email,wc_is_paying,wc_role,wc_user_created,wc_user_id,wc_products,wc_item_total,wc_payments,wc_ip_address
0,amz_local_search.search,2019-10-09 04:09:44.605,NaT,1-S32M4kGRT6ps7uUJzKygVg,1,None,False,coloring book,None,None,...,2019-10,NaN,NaN,NaN,NaN,NaN,"Subscription - Basic, Feature: ASINs included ...",141.5,"[25.0, 0.0, 12.5, 5.0, 99.0, 0.0]",24.19.170.196


In [27]:
print('Unmapped user id-s:')
full_actions_df[pd.isnull(full_actions_df.wc_email)].user_id.unique()

Unmapped user id-s:


array([    1,     2, 17245, 17277, 17278, 17301, 17530, 17856, 17345,
       18031, 18045, 18235, 18287])

# Insights

In [28]:
actions_per_user_df = full_actions_df.groupby('user_id').agg({
        'wc_email': { 'wc_email': 'last' },
        'wc_is_paying': { 'wc_is_paying': 'last' },
        'arg_query': { 'num_keywords': 'nunique' }, 
        'arg_asin': { 'num_asins': 'nunique' }, 
        'occurred_at_month': { 'num_months_active': 'nunique' },    
        'occurred_at_week': { 'num_weeks_active': 'nunique' },
        'occurred_at_day': { 'num_days_active': 'nunique', 'last_day_seen': 'last' },
        'action_id': { 'num_requests': 'count' },
        'wc_user_created': { 'registered_at': 'last' },
        'wc_products': { 'orders': 'last' },
        'wc_item_total': { 'orders_total': 'last' },
        'wc_payments': { 'payments': 'last' },
        'wc_ip_address': { 'ip_address':  'last' }
    })

actions_per_user_df.columns = actions_per_user_df.columns.droplevel(0)
actions_per_user_df.sort_values(['num_weeks_active', 'num_days_active'], ascending=[False, False], inplace=True)
actions_per_user_df.head(100)

,num_weeks_active,num_months_active,registered_at,last_day_seen,num_days_active,orders,wc_is_paying,ip_address,payments,num_asins,orders_total,wc_email,num_keywords,num_requests
user_id,,,,,,,,,,,,,,
17457,91,23,2019-11-05T08:20:46,2021-09-20,436,"Feature: ASINs included in keyword - 10 ASINs,...",True,79.183.98.7,"[12.75, 8.5, 12.75, 8.5, 12.75, 8.5, 12.75, 8....",92,488.75,Hen@Expander.co,446,47835
17827,73,18,2020-03-12T12:25:02,2021-08-09,334,"Feature: ASINs included in keyword - 10 ASINs,...",True,12.186.143.218,"[858.0, 858.0, 10.0, 99.0]",206,1825.00,TRADERMIKIAL@GMAIL.COM,318,113374
17871,72,18,2020-04-12T17:23:47,2021-09-19,328,Subscription - Basic,True,73.138.65.214,"[25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25....",4,413.75,investxol@outlook.com,7,4163
17376,59,19,2019-10-25T14:55:57,2021-04-19,114,Subscription (Annual) - Basic,True,73.240.11.122,"[210.0, 210.0]",34,420.00,brian.exile@gmail.com,32,1273
17901,58,16,2020-04-27T14:07:48,2021-09-17,235,"Subscription - Basic, Feature: ASINs included ...",True,142.129.122.131,"[10.0, 25.0, 10.0, 25.0, 10.0, 25.0, 10.0, 25....",5,553.00,amzjet@kagaco.com,26,4541
17856,58,16,NaN,2021-07-24,168,Subscription - Starter,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",27,15.00,NaN,58,8263
17500,57,19,2019-11-17T08:04:56,2021-07-12,104,"Subscription - Basic, Feature: ASINs included ...",True,74.88.44.149,"[21.25, 8.5, 21.25, 8.5, 21.25, 8.5, 21.25, 8....",102,1154.50,moshegarfinkel@gmail.com,92,4173
17929,53,17,2020-05-18T16:43:23,2021-09-14,175,"Feature: ASINs included in keyword - 10 ASINs,...",True,68.227.62.34,"[15.0, 10.0, 15.0, 10.0, 15.0, 10.0, 15.0, 10....",76,425.00,mattaltman89@gmail.com,41,2003
17413,41,11,2019-10-28T02:28:58,2020-09-27,261,"Feature: Volatility, variability, Subscription...",True,136.49.90.82,"[200.0, 49.0, 200.0, 49.0, 200.0, 49.0, 200.0,...",38,2093.00,tkillian@connectivonline.com,227,320507


In [29]:
actions_per_user_df[[
    'wc_email', 'wc_is_paying',
    'last_day_seen', 'num_keywords', 'num_asins',
    'num_days_active', 'num_weeks_active',
    'num_months_active', 'num_requests',
    'registered_at',
    'orders_total',
    'orders',
    'payments',
    'ip_address']].to_csv(os.path.join(OUT_DIR, 'all_actions.csv'))

In [30]:
# TODO
# 1. Correlate users by asins
# 2. Compure MRR and churn
# 3. Add time since registration
# 4. Visualize registrations

# Same IP

In [31]:
actions_per_user_df.groupby('ip_address').agg({'wc_email': 'nunique' }).sort_values(by='wc_email', ascending=False)

,wc_email
ip_address,
184.65.133.19,4
108.51.61.94,2
69.123.220.152,1
69.148.245.26,1
69.216.18.51,1
70.169.227.87,1
70.182.6.26,1
70.187.226.122,1
70.190.113.252,1


In [32]:
actions_per_user_df[actions_per_user_df.ip_address=='184.65.133.19']

,num_weeks_active,num_months_active,registered_at,last_day_seen,num_days_active,orders,wc_is_paying,ip_address,payments,num_asins,orders_total,wc_email,num_keywords,num_requests
user_id,,,,,,,,,,,,,,
17517,27,13,2019-11-20T20:25:03,2021-09-07,56,"Subscription - Basic, Subscription (Trial only...",True,184.65.133.19,"[15.0, 15.0, 15.0, 0.0]",17,45.0,inquiry@hyponix.net,7,585
17616,2,1,2019-12-15T21:27:37,2019-12-26,4,Subscription (Trial only) - Basic,True,184.65.133.19,"[25.0, 0.0]",2,25.0,mohijoj796@tmailer.org,4,49
17731,2,1,2020-01-23T18:49:08,2020-01-31,3,Subscription (Trial only) - Basic,True,184.65.133.19,"[25.0, 0.0]",3,25.0,mavodix550@it-smart.org,16,99
17609,1,1,2019-12-13T23:36:49,2019-12-15,2,Subscription (Trial only) - Basic,True,184.65.133.19,"[25.0, 0.0]",1,25.0,pebiv71186@tmailer.org,5,31
